In [1]:
import cv2
import numpy as np

img_o = cv2.imread("mona_lisa.jpg")
img = cv2.cvtColor(img_o, cv2.COLOR_BGR2LAB )

In [2]:
HEIGHT, WIDTH, _ = img.shape

In [3]:
def crop(img, bounding_box):
    return img[bounding_box[1]:bounding_box[3], bounding_box[0]:bounding_box[2]]

In [4]:
def find_cut(img, bounding_box, direction="x", depth=5, t=5, failed_previous=False):
   
    x_min = bounding_box[0]
    x_max = bounding_box[2]
    y_min = bounding_box[1]
    y_max = bounding_box[3]
    
    if x_max - x_min < 20 and y_max - y_min < 20:
        if x_max == x_min or y_max == y_min:
            return None
        else:
            answer = ((x_min, y_min, x_max, y_max), )
            assert type(answer[0]) == tuple or answer == None
            return answer
    
    possibilities = []
    for k in range(depth):
        p = (k+1)/(depth+1)
        if direction == "x":
            l = (x_min, y_min, int(x_min+(x_max-x_min)*p), y_max)
            r = (int(x_min+(x_max-x_min)*p), y_min, x_max, y_max)
            possibilities.append((l, r))
        elif direction == "y":
            l = (x_min, y_min, x_max, int(y_min+(y_max-y_min)*p))
            r = (x_min, int(y_min+(y_max-y_min)*p), x_max, y_max)
            possibilities.append((l, r))
        else:
            raise ValueError("Incorrect direction")
            
    m = 0
    b = possibilities[0]
    for k in possibilities:
        l = crop(img, k[0])
        r = crop(img, k[1])
        l_c = l.mean(axis=0).mean(axis=0)
        r_c = r.mean(axis=0).mean(axis=0)
        d = np.linalg.norm(l_c-r_c)
        if d>m:
            m = d
            b = k
            
    if m < t:
        answer = ((x_min, y_min, x_max, y_max), )
        assert type(answer[0] == tuple)
        if failed_previous == True:
            return answer
        else:
            if direction == 'y':
                new_direction = 'x'
            else:
                new_direction = 'y'
            return find_cut(img, bounding_box, new_direction, depth=depth, failed_previous=True) 
    if depth == 1:
        assert type(b[0] == tuple)
        return b
    else:
        if direction == 'y':
            new_direction = 'x'
        else:
            new_direction = 'y'
        a1 = find_cut(img, b[0], new_direction, depth=depth-1) 
        a2 = find_cut(img, b[1], new_direction, depth=depth-1)
        assert type(a1[0]) == tuple or a1 == None
        assert type(a2[0]) == tuple or a2 == None
        if a1 is not None:
            if a2 is not None:
                answer = a1+a2
            else:
                answer = a1
        else:
            if a2 is not None:
                answer = a2
            else:
                answer = None
        assert type(answer[0]) == tuple or answer == None
        return answer

In [5]:
cuts = find_cut(img, (0, 0, WIDTH, HEIGHT), depth=17, t=15)

<ipython-input-4-10188bd4fbcb>:35: RuntimeWarning: Mean of empty slice.
  l_c = l.mean(axis=0).mean(axis=0)
C:\Users\mateu\anaconda3\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [6]:
art = np.zeros((HEIGHT,WIDTH,3), np.uint8)
for cut in cuts:
    art = cv2.rectangle(art, pt1=(cut[0], cut[1]), pt2=(cut[2], cut[3]), color=crop(img_o, cut).mean(axis=0).mean(axis=0), thickness=-1)

In [7]:
cv2.imshow("Image", np.concatenate((art, img_o), axis=1))
cv2.waitKey(0) 

cv2.destroyAllWindows()

In [8]:
cv2.imwrite("3.jpg", np.concatenate((art, img_o), axis=1))

True